In [ ]:
import numpy as np
from scipy.stats import pearsonr
from sklearn import datasets
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm
import statsmodels.formula.api as smf
from sklearn.preprocessing import StandardScaler 
from statsmodels.iolib.summary2 import summary_col
from statsmodels.api import add_constant
from sklearn.decomposition import PCA
from sklearn import decomposition
from sklearn import datasets
from sklearn.preprocessing import scale
from mlxtend.preprocessing import MeanCenterer
from textwrap import wrap

# Aim 1: Do components of Prospect-Refuge Theory predict preference?

> Indented block




Prospect Refuge Theory proxies
*   Transparency (prospect)
*   Enclosure (refuge) 
*   Complexity (complexity)
*   Imageability (mystery)



## Step 1. Import, Organize and Clean

In [ ]:
scaler = StandardScaler() #will normalize the features (each column of X, INDIVIDUALLY) so that each feature will have mean = 0 and standard deviation = 1.

### Importing ratings of street-view images and only pulling data needed for prt and preference

In [ ]:
rates = pd.read_csv("https://raw.githubusercontent.com/gabyakcelik/Street_Psych_Final/mainbranch/scoreTable_2.csv")

In [ ]:
pref = rates['Preference']
proxies = rates[['Transparency', 'Enclosure', 'Complexity', 'Imageability']]
prefproxy = rates[['Preference', 'Transparency', 'Enclosure', 'Complexity', 'Imageability']]

In [ ]:
#from google.colab import files

#const_low.to_csv('clean_llvf.csv', encoding = 'utf-8-sig') 
#files.download('clean_llvf.csv')

In [ ]:
mlvf = pd.read_csv("https://raw.githubusercontent.com/gabyakcelik/Street_Psych_Final/mainbranch/googlestreetviewimgs_feat_extract/sw_mlvf_6.7.2021.csv")

In [ ]:
mlvf

In [ ]:
sns.histplot(mlvf['pcnt_vegetation'])

#### z-scoring the proxies selected for PRT and adding a constant

In [ ]:
stan_proxies = scaler.fit_transform(proxies)
stan_proxies = sm.add_constant(stan_proxies)
stan_proxies = pd.DataFrame(stan_proxies)
stan_proxies.columns = ['Const','Transparency', 'Enclosure', 'Complexity', 'Imageability']
stan_proxies

In [ ]:
stan_prefproxy = scaler.fit_transform(prefproxy)
stan_prefproxy = sm.add_constant(stan_prefproxy)
stan_prefproxy = pd.DataFrame(stan_prefproxy)
stan_prefproxy.columns = ['Const','Preference', 'Transparency', 'Enclosure', 'Complexity', 'Imageability']
stan_prefproxy

### Importing low-level features and concatenating straight edge density data with other low-level features. Then,  taking the first 552 images to use in our data. 

In [ ]:
llvf_nosed = pd.read_csv('https://raw.githubusercontent.com/gabyakcelik/Street_Psych_Final/mainbranch/googlestreetviewimgs_feat_extract/sw_LLVF_6.7.2021.csv')
llvf_nsedsed = pd.read_csv('https://raw.githubusercontent.com/gabyakcelik/Street_Psych_Final/mainbranch/googlestreetviewimgs_feat_extract/sw_ED_6.7.2021.csv')
llvf = pd.concat([llvf_nosed, llvf_nsedsed], axis=1)
llvf = llvf.head(552)
llvf = llvf.drop(['ED', 'id'], axis=1)
llvf.head()

#### Adding constant to low-level features, data is already z-scored coming in 

In [ ]:
const_low = sm.add_constant(llvf)
const_low

In [ ]:
from google.colab import files

const_low.to_csv('clean_llvf.csv', encoding = 'utf-8-sig') 
files.download('clean_llvf.csv')

### Importing mid-level feature data and organizing by putting image numbers in order, and then taking the first 552 images to use in our data. 

In [ ]:
mlvf = pd.read_csv('https://raw.githubusercontent.com/gabyakcelik/street_psych/master/ReExtracted_Data_6.7.2021/sw_mlvf_6.7.2021.csv')
mlvf = mlvf.sort_values(by =['image_name']) 
mlvf = mlvf.reset_index(drop=True)
mlvf = pd.DataFrame(mlvf)
mlvf.columns = ['id','Road', 'Sidewalk', 'Building', 
	'Wall', 'Fence', 'Pole', 'Traffic light', 'Traffic sign', 'Vegetation', 'Terrain', 'Sky', 
	'Person', 'Rider', 'Car', 'Truck', 'Bus', 'Train', 'Motorcycle', 'Bicycle', 'Others']
mlvf = mlvf.head(552)
mlvf

In [ ]:
mlvf = mlvf[['Road', 'Sidewalk', 'Building', 
	'Wall', 'Fence', 'Pole', 'Traffic light', 'Traffic sign', 'Vegetation', 'Terrain', 'Sky', 
	'Person', 'Rider', 'Car', 'Truck', 'Bus', 'Train', 'Motorcycle', 'Bicycle', 'Others']]

#### z-scoring the mid-level features and additng a constant

In [ ]:
stan_mid = scaler.fit_transform(mlvf)
stan_mid = sm.add_constant(stan_mid)
stan_mid = pd.DataFrame(stan_mid)
stan_mid.columns =['Const','Road', 'Sidewalk', 'Building', 
	'Wall', 'Fence', 'Pole', 'Traffic light', 'Traffic sign', 'Vegetation', 'Terrain', 'Sky', 
	'Person', 'Rider', 'Car', 'Truck', 'Bus', 'Train', 'Motorcycle', 'Bicycle', 'Others']
stan_mid

In [ ]:
from google.colab import files

stan_mid.to_csv('clean_mlvf.csv', encoding = 'utf-8-sig') 
files.download('clean_mlvf.csv')

#### Heat Map for Mid-level features, to see how correlated they are with one another

In [ ]:
title='Corrleation Matrix for Mid-Level Features'

sns.set_theme(style='darkgrid')
plt.figure(figsize=(40, 40))
sns.set(font_scale=1.5)
p = sns.heatmap(data=mlvf.corr(), annot=True)
p.set_xticklabels(p.get_xticklabels(),rotation=45)
p.set_yticklabels(p.get_yticklabels(),rotation=45)
p.tick_params(labelsize=24)
plt.title(title, fontsize=70)
plt.show()

## Step 2. Conducting a correlation plot to show relationships between each of the PRT proxies and preference.

## Function to display r-value onto each correlation plot

In [ ]:
def corrfunc(x,y, ax=None, **kws):
#    """Plot the correlation coefficient in the top left corner of a plot."""
    r, _ = pearsonr(x, y)
    ax = ax or plt.gca()
    corrcoeff = 'r'
    ax.annotate(f'{corrcoeff} = {r:.2f}', xy=(.1, .9), xycoords=ax.transAxes)

### Code for the correlation plot

In [ ]:
sns.set_theme(style='darkgrid')
r = sns.pairplot(prefproxy, kind='reg', diag_kind="kde", plot_kws={'line_kws':{'color':'black'}, 'scatter_kws': {'alpha': 0.5}})
r.set(xlim=(0.0,1.0), ylim =(0.0,1.0))
r.map_upper(corrfunc)
r.map_lower(corrfunc)
r.map_diag(sns.distplot)
plt.show()

## Step 3. Conducting linear regression to see which PRT proxies are predictive of preference.

## Code for linear regression 

In [ ]:
proxyprefreg = sm.OLS(rates['Preference'], proxies).fit()
print(proxyprefreg.summary())

### Code for visualizing beta values from regression

In [ ]:
proxypref_betas= pd.DataFrame(proxyprefreg.params)
proxypref_betas = proxypref_betas.T
proxypref_betas

In [ ]:
proxypref_sigbetas = pd.DataFrame(proxypref_betas[['Transparency', 'Enclosure', 'Complexity', 'Imageability']])
proxypref_sigbetas

In [ ]:
sns.set_theme(style='darkgrid')
plt.figure(figsize=(23, 10))
p = sns.barplot(data=proxypref_sigbetas)
p.set_xticklabels(p.get_xticklabels(),rotation=45)
p.tick_params(labelsize=24)
p.set_title('Visualizing Significant Betas for PRT Proxies That Predict Preference', fontdict= { 'fontsize': 24, 'fontweight':'bold'})
plt.show()

# Aim 2: Which mid-level features are associated with each of the PRT proxies?

## Step 1. Conduct a regression to see what mid-level features predict Transparency (Prospect)

In [ ]:
mid_trans_reg = sm.OLS(rates['Transparency'], stan_mid).fit()
print(mid_trans_reg.summary())

### Visualizing significant betas from above regression

In [ ]:
mid_trans_betas= pd.DataFrame(mid_trans_reg.params).drop(['Const'], axis=0)
mid_trans_betas = mid_trans_betas.T
mid_trans_betas

In [ ]:
mid_trans_sigbetas = pd.DataFrame(mid_trans_betas[['Road', 'Wall', 'Traffic light', 'Traffic sign', 'Vegetation', 'Terrain', 'Sky', 
	'Person', 'Car', 'Bicycle']])
mid_trans_sigbetas

In [ ]:
title='Visualizing Significant Betas for Mid-Level Features That Predict Transparency (Prospect)'

sns.set_theme(style='darkgrid')
plt.figure(figsize=(10, 10))
p = sns.barplot(data=mid_trans_sigbetas)
p.set_xticklabels(p.get_xticklabels(),rotation=45)
p.tick_params(labelsize=24)
plt.title('\n'.join(wrap(title,35)), fontsize=24)
plt.show()

## Step 2. Conduct a regression to see what mid-level features predict Enclosure (Refuge)

In [ ]:
mid_encl_reg = sm.OLS(rates['Enclosure'], stan_mid).fit()
print(mid_encl_reg.summary())

### Visualizing significant betas from above regression

In [ ]:
mid_encl_betas = pd.DataFrame(mid_encl_reg.params).drop(['Const'], axis=0)
mid_encl_betas = mid_encl_betas.T 
mid_encl_betas

In [ ]:
mid_encl_sigbetas = pd.DataFrame(mid_encl_betas[['Sidewalk','Building', 'Pole', 'Vegetation', 'Sky', 'Person', 'Others']])

In [ ]:
title='Visualizing Significant Betas for Mid-Level Features That Predict Enclosure(Refuge)'

sns.set_theme(style='darkgrid')
plt.figure(figsize=(10, 10))
p = sns.barplot(data=mid_encl_sigbetas)
p.set_xticklabels(p.get_xticklabels(),rotation=45)
p.tick_params(labelsize=24)
plt.title('\n'.join(wrap(title,35)), fontsize=24)
plt.show()

## Step 3. Conduct a regression to see what mid-level features predict Complexity (Complexity)

In [ ]:
mid_comp_reg = sm.OLS(rates['Complexity'], stan_mid).fit()
print(mid_comp_reg.summary())

### Visualizing significant betas from above regression

In [ ]:
mid_comp_betas= pd.DataFrame(mid_comp_reg.params).drop(['Const'], axis=0)
mid_comp_betas = mid_comp_betas.T
mid_comp_betas

In [ ]:
mid_comp_sigbetas = pd.DataFrame(mid_comp_betas[['Road','Building', 'Wall', 'Traffic light', 'Vegetation', 'Sky', 'Person', 'Car', 'Bicycle', 'Others']])

In [ ]:
title='Visualizing Significant Betas for Mid-Level Features That Predict Complexity'

sns.set_theme(style='darkgrid')
plt.figure(figsize=(10, 10))
p = sns.barplot(data=mid_comp_sigbetas)
p.set_xticklabels(p.get_xticklabels(),rotation=45)
p.tick_params(labelsize=24)
plt.title('\n'.join(wrap(title,35)), fontsize=24)
plt.show()

## Step 4. Conduct a regression to see what mid-level features predict Imageability (Mystery)

In [ ]:
mid_img_reg = sm.OLS(rates['Imageability'], stan_mid).fit()
print(mid_img_reg.summary())

### Visualizing significant betas from above regression

In [ ]:
mid_img_betas= pd.DataFrame(mid_img_reg.params).drop(['Const'], axis=0)
mid_img_betas = mid_img_betas.T
mid_img_betas

In [ ]:
mid_img_sigbetas = pd.DataFrame(mid_img_betas[['Wall', 'Traffic light', 'Vegetation', 'Sky', 'Person', 'Others']])

In [ ]:
title='Visualizing Significant Betas for Mid-Level Features That Predict Imageability(Mystery)'

sns.set_theme(style='darkgrid')
plt.figure(figsize=(10, 10))
p = sns.barplot(data=mid_img_sigbetas)
p.set_xticklabels(p.get_xticklabels(),rotation=45)
p.tick_params(labelsize=24)
plt.title('\n'.join(wrap(title,35)), fontsize=24)
plt.show()

# Aim 3: Do mid-level features that contribute to PRT predict preference?

## Step 1. Conduct regression to see if mid-level features are predictive of preference. 

In [ ]:
mid_pref_reg = sm.OLS(rates['Preference'], stan_mid).fit()
print(mid_pref_reg.summary())

### Visualizing significant betas from above regression

In [ ]:
mid_pref_betas= pd.DataFrame(mid_pref_reg.params).drop(['Const'], axis=0)
mid_pref_betas = mid_pref_betas.T
mid_pref_betas

In [ ]:
mid_pref_sigbetas = pd.DataFrame(mid_pref_betas[['Building', 'Wall', 'Traffic light', 'Vegetation', 'Sky', 
	'Person', 'Truck']])
mid_pref_sigbetas

In [ ]:
sns.set_theme(style='darkgrid')
plt.figure(figsize=(23, 10))
p = sns.barplot(data=mid_pref_sigbetas)
p.set_xticklabels(p.get_xticklabels(),rotation=45)
p.tick_params(labelsize=24)
p.set_title('Visualizing Significant Betas for Mid-Level Features That Predict Preference', fontdict= { 'fontsize': 24, 'fontweight':'bold'})
plt.show()

# Aim 4: Do the PRT proxies mediate the relationship between mid-level features and preference?


The MLVF significantly associated with prefernce are:


1.   Building
2.   Wall
3.   Traffic Light
4.   Vegetation 
5.   Sky
6.   Person 
7.   Truck


## Step 1. Understand the mediation function

```
pingouin.mediation_analysis(data=None, x=None, m=None, y=None, covar=None, alpha=0.05, n_boot=500, seed=None, return_dist=False)
```
> Parameters:
>  *   x: str - Column name in data containing the predictor variable. The predictor variable must be continuous.
>  *   m: str or list of str - Column name(s) in data containing the mediator variable(s). The mediator(s) can be continuous or binary (e.g. 0 or 1). This function supports multiple parallel mediators.
>  *   y: str  - Column name in data containing the outcome variable. The outcome variable must be continuous
>  *   covar: None, str or list - Covariate(s). If not None, the specified covariate(s) will be included in all regressions.
>  *   alpha: float - Significance threshold. Used to determine the confidence interval, CI = [a/2; 1-a/2]
>  *   n_boot: int  - Number of bootstrap iterations for confidence intervals and p-values estimation. The greater, the slower
>  *   seed: int or None  - Random state seed.
>  *   return_dist: bool - If True, the function also returns the indirect bootstrapped beta samples (size = n_boot). Can be plotted for instance using `seaborn.distplot()` or `seaborn.kdeplot()` functions.


> Returns:
Mediation Summary
>  *   `path`: regression model
>  *   `coef`: regression estimates
>  *   `se`: standard error
>  *   `CI[2.5%]`: lower confidence interval
>  *   `CI[97.5%]`: upper confidence interval
>  *   `pval`: two-sided p-values
>  *   `sig`: statistical significance

_from: https://pingouin-stats.org/generated/pingouin.mediation_analysis.html_

Step 1: Look at Total Effect - this will tell you if the IV and DV are related to each other.

Step 2: Look at the effect of the IV on the mediator - in order for a mediation to take place, an IV must affect the mediator, or else the mediator being tested is not valid.

Step 3: Look at the effect of the mediator on the DV, controlling for the IV - indirect effect, the effect of IV on the DV thru the mediator

Step 4: Compare the direct effect to the indirect effect - proportion mediated, which is dividing the indirect by the total effect (describes the proportion of the effect of the IV on the DV that goes through the mediator. )

_from: https://towardsdatascience.com/doing-and-reporting-your-first-mediation-analysis-in-r-2fe423b92171_




## Step 2: Conduct Seperate Mediations for each mid-level feature that significantly predicted preference for each of the PRT proxies.

### Install and import the mediation function detailed above.

In [ ]:
!pip install pingouin

In [ ]:
from pingouin import mediation_analysis

### Making a dataframe to use for mediation analysis

In [ ]:
medpref = pd.concat([rates['Preference'], stan_mid], axis=1)

In [ ]:
medprt = pd.concat([stan_proxies, medpref], axis=1)
medprt.head()
medprt.columns

## Step 2.1: Mediations for Building

### Mediation: Building and Transparency

In [ ]:
buildmedtr = mediation_analysis(data=medprt, x='Building', m='Transparency', y='Preference', alpha=0.05, seed=42).round(3)
buildmedtr

#### Proportion Mediated

In [ ]:
propmed_buildtr = buildmedtr.coef[4]/buildmedtr.coef[2]
propmed_buildtr

### Mediation: Building and Enclosure

In [ ]:
buildmedencl = mediation_analysis(data=medprt, x='Building', m='Enclosure', y='Preference', alpha=0.05, seed=42).round(3)
buildmedencl

#### Propotion Mediated

In [ ]:
propmed_buildmedencl = buildmedencl.coef[4]/buildmedencl.coef[2]
propmed_buildmedencl

### Mediation: Building and Complexity

In [ ]:
buildmedcomp = mediation_analysis(data=medprt, x='Building', m='Complexity', y='Preference', alpha=0.05, seed=42).round(3)
buildmedcomp

#### Propotion Mediated

In [ ]:
propmed_buildmedcomp = buildmedcomp.coef[4]/buildmedcomp.coef[2]
propmed_buildmedcomp

### Mediation: Building and Imageability

In [ ]:
buildmedimg = mediation_analysis(data=medprt, x='Building', m='Imageability', y='Preference', alpha=0.05, seed=42).round(3)
buildmedimg

#### Propotion Mediated

In [ ]:
propmed_buildmedimg = buildmedimg.coef[4]/buildmedimg.coef[2]
propmed_buildmedimg

## Step 2.2: Mediations for Wall 

### Mediation: Wall and Transparency

In [ ]:
wallmedtr = mediation_analysis(data=medprt, x='Wall', m='Transparency', y='Preference', alpha=0.05, seed=42).round(3)
wallmedtr 

#### Proportion Mediated

In [ ]:
propmed_wallmedtr = wallmedtr.coef[4]/wallmedtr.coef[2]
propmed_wallmedtr

### Mediation: Wall and Enclosure

In [ ]:
wallmedencl = mediation_analysis(data=medprt, x='Wall', m='Enclosure', y='Preference', alpha=0.05, seed=42).round(3)
wallmedencl

#### Propotion Mediated

In [ ]:
propmed_wallmedencl = wallmedencl.coef[4]/wallmedencl.coef[2]
propmed_wallmedencl

### Mediation: Wall and Complexity

In [ ]:
wallmedcomp = mediation_analysis(data=medprt, x='Wall', m='Complexity', y='Preference', alpha=0.05, seed=42).round(3)
wallmedcomp

#### Propotion Mediated

In [ ]:
propmed_buildmedcomp = buildmedcomp.coef[4]/buildmedcomp.coef[2]
propmed_buildmedcomp

### Mediation: Wall and Imageability

In [ ]:
buildmedimg = mediation_analysis(data=medprt, x='Building', m='Imageability', y='Preference', alpha=0.05, seed=42).round(3)
buildmedimg

#### Propotion Mediated

In [ ]:
propmed_buildmedimg = buildmedimg.coef[4]/buildmedimg.coef[2]
propmed_buildmedimg

## Step 2.3: Mediations for Traffic Light


### Mediation: Traffic Light and Transparency

In [ ]:
tlmedtr = mediation_analysis(data=medprt, x='Traffic light', m='Transparency', y='Preference', alpha=0.05, seed=42).round(3)
tlmedtr

#### Proportion Mediated

In [ ]:
propmed_tlmedtr = tlmedtr.coef[4]/tlmedtr.coef[2]
propmed_tlmedtr 

### Mediation: Traffic Light and Enclosure

In [ ]:
tlmedencl = mediation_analysis(data=medprt, x='Traffic light', m='Enclosure', y='Preference', alpha=0.05, seed=42).round(3)
tlmedencl

#### Proportion Mediated

In [ ]:
propmed_tlmedencl = tlmedencl.coef[4]/tlmedencl.coef[2]
propmed_tlmedencl

### Mediation: Traffic Light and Complexity

In [ ]:
tlmedcomp = mediation_analysis(data=medprt, x='Traffic light', m='Complexity', y='Preference', alpha=0.05, seed=42).round(3)
tlmedcomp

#### Proportion Mediated

In [ ]:
propmed_tlmedcomp = tlmedcomp.coef[4]/tlmedcomp.coef[2]
propmed_tlmedcomp

### Mediation: Traffic Light and Imageability

In [ ]:
tlmedimg = mediation_analysis(data=medprt, x='Traffic light', m='Imageability', y='Preference', alpha=0.05, seed=42).round(3)
tlmedimg

#### Proportion Mediated

In [ ]:
propmed_tlmedimg = tlmedimg.coef[4]/tlmedimg.coef[2]
propmed_tlmedimg

## Step 2.4: Mediations for Vegetation

### Mediation: Vegetation and Transparency

#### Proportion Mediated

### Mediation: Vegetation and Enclosure

#### Proportion Mediated

### Mediation: Vegetation and Complexity 

#### Proportion Mediated

### Mediation: Vegetation and Imageability

#### Proportion Mediated

## Steps 2.5: Mediations for Sky

### Mediation: Sky and Transparency

In [ ]:
skymedtr = mediation_analysis(data=medprt, x='Sky', m='Transparency', y='Preference', alpha=0.05, seed=42).round(3)
skymedtr

#### Proportion Mediated

In [ ]:
propmed_skymedtr = skymedtr.coef[4]/skymedtr.coef[2]
propmed_skymedtr

### Mediation: Sky and Enclosure

In [ ]:
skymedencl = mediation_analysis(data=medprt, x='Sky', m='Enclosure', y='Preference', alpha=0.05, seed=42).round(3)
skymedencl

#### Proportion Mediated

In [ ]:
propmed_skymedencl = skymedencl.coef[4]/skymedencl.coef[2]
propmed_skymedencl

### Mediation: Sky and Complexity

In [ ]:
skymedcomp = mediation_analysis(data=medprt, x='Sky', m='Complexity', y='Preference', alpha=0.05, seed=42).round(3)
skymedcomp

#### Proportion Mediated

In [ ]:
propmed_skymedcomp = skymedcomp.coef[4]/skymedcomp.coef[2]
propmed_skymedcomp

### Mediation: Sky and Imageability

In [ ]:
skymedimg = mediation_analysis(data=medprt, x='Sky', m='Imageability', y='Preference', alpha=0.05, seed=42).round(3)
skymedimg

#### Proportion Mediated

In [ ]:
propmed_skymedimg = skymedimg.coef[4]/skymedimg.coef[2]
propmed_skymedimg

## Steps 2.5: Mediations for Person

### Mediation: Person and Transparency

In [ ]:
permedtr = mediation_analysis(data=medprt, x='Person', m='Transparency', y='Preference', alpha=0.05, seed=42).round(3)
permedtr

#### Proportion Mediated

In [ ]:
propmed_permedtr = permedtr.coef[4]/permedtr.coef[2]
propmed_permedtr

### Mediation: Person and Enclosure

In [ ]:
permedencl = mediation_analysis(data=medprt, x='Person', m='Enclosure', y='Preference', alpha=0.05, seed=42).round(3)
permedencl

#### Proportion Mediated

In [ ]:
propmed_permedencl = permedencl.coef[4]/permedencl.coef[2]
propmed_permedencl

### Mediation: Person and Complexity

In [ ]:
permedcomp = mediation_analysis(data=medprt, x='Person', m='Complexity', y='Preference', alpha=0.05, seed=42).round(3)
permedcomp

#### Proportion Mediated

In [ ]:
propmed_permedcomp = permedcomp.coef[4]/permedcomp.coef[2]
propmed_permedcomp

## Mediation: Person and Imageability

In [ ]:
permedimg = mediation_analysis(data=medprt, x='Person', m='Imageability', y='Preference', alpha=0.05, seed=42).round(3)
permedimg

#### Proportion Mediated 

In [ ]:
propmed_permedimg = permedimg.coef[4]/permedimg.coef[2]
propmed_permedimg

## Steps 2.6: Mediations for Truck

### Mediation: Truck and Transparency

In [ ]:
truckmedtr = mediation_analysis(data=medprt, x='Truck', m='Transparency', y='Preference', alpha=0.05, seed=42).round(3)
truckmedtr

#### Proportion Mediated

In [ ]:
propmed_truckmedtr = truckmedtr.coef[4]/truckmedtr.coef[2]
propmed_truckmedtr

### Mediation: Truck and Enclosure

In [ ]:
truckmedencl = mediation_analysis(data=medprt, x='Truck', m='Enclosure', y='Preference', alpha=0.05, seed=42).round(3)
truckmedencl

#### Proportion Mediated

In [ ]:
propmed_truckmedencl = truckmedencl.coef[4]/truckmedencl.coef[2]
propmed_truckmedencl

### Mediation: Truck and Complexity

In [ ]:
truckmedcomp = mediation_analysis(data=medprt, x='Truck', m='Complexity', y='Preference', alpha=0.05, seed=42).round(3)
truckmedcomp

#### Proportion Mediated

In [ ]:
propmed_truckmedcomp = truckmedcomp.coef[4]/truckmedcomp.coef[2]
propmed_truckmedcomp

### Mediation: Truck and Imageability

In [ ]:
truckmedimg = mediation_analysis(data=medprt, x='Truck', m='Imageability', y='Preference', alpha=0.05, seed=42).round(3)
truckmedimg

#### Proportion Mediated

In [ ]:
propmed_truckmedimg = truckmedimg.coef[4]/truckmedimg.coef[2]
propmed_truckmedimg

## Step 3. Conduct mediation analyses with each PRT proxy per mid-level feature

### Step 3.1 Mediation for Building

In [ ]:
mediation_analysis(data=medprt, x='Building', m=['Transparency', 'Enclosure', 'Complexity', 'Imageability'], y='Preference', alpha=0.05,
                   seed=42).round(3)

#### Multiple Comparisons, survive 0.007

In [ ]:
mediation_analysis(data=medprt, x='Building', m=['Transparency', 'Enclosure', 'Complexity', 'Imageability'], y='Preference', alpha=0.007,
                   seed=42).round(3)

### Step 3.2 Mediation for Wall

In [ ]:
mediation_analysis(data=medprt, x='Wall', m=['Transparency', 'Enclosure', 'Complexity', 'Imageability'], y='Preference', alpha=0.05,
                   seed=42).round(3)

#### Multipe Comparisons, survive 0.007

In [ ]:
mediation_analysis(data=medprt, x='Wall', m=['Transparency', 'Enclosure', 'Complexity', 'Imageability'], y='Preference', alpha=0.007,
                   seed=42).round(3)

### Step 3.3 Mediation for Traffic Light

In [ ]:
mediation_analysis(data=medprt, x='Traffic light', m=['Transparency', 'Enclosure', 'Complexity', 'Imageability'], y='Preference', alpha=0.05,
                   seed=42).round(3)

#### Multiple Comparisons, survive 0.007


### Step 3.4 Mediation for Vegetation

In [ ]:
mediation_analysis(data=medprt, x='Vegetation', m=['Transparency', 'Enclosure', 'Complexity', 'Imageability'], y='Preference', alpha=0.05,
                   seed=42).round(3)

#### Multiple Comparisons, survive 0.007


In [ ]:
mediation_analysis(data=medprt, x='Vegetation', m=['Transparency', 'Enclosure', 'Complexity', 'Imageability'], y='Preference', alpha=0.007,
                   seed=42).round(3)

### Step 3.5 Mediation for Sky

In [ ]:
mediation_analysis(data=medprt, x='Sky', m=['Transparency', 'Enclosure', 'Complexity', 'Imageability'], y='Preference', alpha=0.05,
                   seed=42).round(3)

#### Multiple Comparisons, survive 0.007

In [ ]:
mediation_analysis(data=medprt, x='Sky', m=['Transparency', 'Enclosure', 'Complexity', 'Imageability'], y='Preference', alpha=0.007,
                   seed=42).round(3)

### Step 3.6 Mediation for Person


In [ ]:
mediation_analysis(data=medprt, x='Person', m=['Transparency', 'Enclosure', 'Complexity', 'Imageability'], y='Preference', alpha=0.05,
                   seed=42).round(3)

#### Multiple Comparisons, survive 0.007

In [ ]:
mediation_analysis(data=medprt, x='Person', m=['Transparency', 'Enclosure', 'Complexity', 'Imageability'], y='Preference', alpha=0.007,
                   seed=42).round(3)

### Step 3.7 Mediation for Truck

In [ ]:
mediation_analysis(data=medprt, x='Truck', m=['Transparency', 'Enclosure', 'Complexity', 'Imageability'], y='Preference', alpha=0.05,
                   seed=42).round(3)

#### Multiple Comparisons, survive 0.007


In [ ]:
mediation_analysis(data=medprt, x='Truck', m=['Transparency', 'Enclosure', 'Complexity', 'Imageability'], y='Preference', alpha=0.007,
                   seed=42).round(3)

# Additional Analyses 

## Importing Memorability data

In [ ]:
mem = pd.read_csv("https://raw.githubusercontent.com/gabyakcelik/Street_Psych_Final/mainbranch/resmem_images_mainside.csv")

## Concatenating imageability and memorability data into one dataframe

In [ ]:
imgby = rates['Imageability']

In [ ]:
memimg = pd.concat([imgby, mem['resmem_pred']],axis=1)

## Conducting a linear regression to see if imageability predicts memorability

### Creating variables and adding constant to prep for linear regression

In [ ]:
x = memimg['Imageability']
y = memimg['resmem_pred']

x = sm.add_constant(x) 
lm = sm.OLS(y, x).fit()

### Conducting regression

In [ ]:
print(lm.summary())

### Visualizing the above regression

In [ ]:
sns.regplot(x="Imageability", y="resmem_pred", data=memimg)

In [ ]:
sns.regplot(x="Imageability", y="resmem_pred", data=memimg, x_estimator=np.mean) 
# ^^^ collapse over the observations in each discrete bin to plot an estimate of central tendency along with a confidence interval

## Correlation between imageability and memorability

In [ ]:
r = sns.pairplot(memimg, kind='reg', diag_kind="kde", plot_kws={'line_kws':{'color':'black'}, 'scatter_kws': {'alpha': 0.5}})
#r.set(xlim=(0.0,1.0), ylim =(0.0,1.0))
r.map_upper(corrfunc)
r.map_lower(corrfunc)
r.map_diag(sns.distplot)
plt.show()

### Heatmap Correlation figure

In [ ]:
sns.heatmap(memimg.corr())
sns.pairplot(memimg)

## Correlation between memorability and all prt proxies


In [ ]:
prtmem = pd.concat([prefproxy, memimg['resmem_pred']], axis=1)

In [ ]:
prtmem 

### Corrplot & reg between prt, preference and memorability


#### Correlation plot

In [ ]:
r = sns.pairplot(prtmem, kind='reg', diag_kind="kde", plot_kws={'line_kws':{'color':'black'}, 'scatter_kws': {'alpha': 0.5}})
r.set(xlim=(0.0,1.0), ylim =(0.0,1.0))
r.map_upper(corrfunc)
r.map_lower(corrfunc)
r.map_diag(sns.distplot)
plt.show()


#### z-scoring values and adding a constant

In [ ]:
stanprtmem = scaler.fit_transform(prtmem)
stanprtmem = sm.add_constant(stanprtmem)
stanprtmem = pd.DataFrame(stanprtmem)
stanprtmem.columns = ['Const','Preference','Transparency', 'Enclosure', 'Complexity', 'Imageability', 'resmem_pred']
stanprtmem = stanprtmem.drop(['resmem_pred'], axis=1)

### Conducting a regression

In [ ]:
prtmemreg = sm.OLS(memimg['resmem_pred'], stanprtmem).fit()
print(prtmemreg.summary())

#### Visualizing significant betas for above regression

In [ ]:
prtmem_betas= pd.DataFrame(prtmemreg.params).drop(['Const'], axis=0)
prtmem_betas = prtmem_betas.T
prtmem_betas

In [ ]:
prtmem_sigbetas = pd.DataFrame(prtmem_betas[['Transparency', 'Imageability']])
prtmem_sigbetas

In [ ]:
title='Visualizing Significant Betas for PRT Proxies & Preference That Predict Memorability'

sns.set_theme(style='darkgrid')
plt.figure(figsize=(10, 10))
p = sns.barplot(data=prtmem_sigbetas)
p.set_xticklabels(p.get_xticklabels(),rotation=45)
p.tick_params(labelsize=24)
plt.title('\n'.join(wrap(title,35)), fontsize=24)
plt.show()

# Creating  a Map of Income and Images 

In [ ]:
!pip install censusdata
!pip install geopandas

In [ ]:
import censusdata
import geopandas as gpd 
import folium
from folium.plugins import MarkerCluster


In [ ]:
#roadgeo = pd.read_csv("https://raw.githubusercontent.com/gabyakcelik/street_psych/master/Geocode/road_geoids_updated.csv")
sidegeo = pd.read_csv("https://raw.githubusercontent.com/gabyakcelik/street_psych/master/Geocode/side_geoids_updated.csv")
census = pd.read_csv("https://raw.githubusercontent.com/gabyakcelik/street_psych/master/Geocode/updated_geoid_census2019.csv")

In [ ]:
sidegeo = sidegeo.drop(['Unnamed: 6'], axis=1)
sidegeo

In [ ]:
census = census.rename(columns={'GEOID_text' : 'block_group_code'})
census

In [ ]:
sw_cen = pd.merge(sidegeo, census, on=['block_group_code'])
sw_cen.head()

In [ ]:
hhi = sw_cen[['a_image_name', 'block_group_code', 'lat', 'lng','MedianHHIncome']]
hhi.head()

In [ ]:
hhi.mean()

In [ ]:
imglocations = sw_cen[['lat', 'lng']]
imglocations
locationlist = imglocations.values.tolist()
len(locationlist)
locationlist[7]

In [ ]:
geo_path = r'/content/Boundaries - Neighborhoods.geojson'
geoJSON_df = gpd.read_file(geo_path)
geoJSON_df

In [ ]:
type(geoJSON_df)
geoJSON_neigh = list(geoJSON_df.sec_neigh.values)
len(geoJSON_neigh)
geoJSON_neigh

In [ ]:
geoJSON_neigh = pd.DataFrame(geoJSON_neigh)
geoJSON_neigh.columns = ['sec_neigh']
geoJSON_neigh

In [ ]:
geoJSON_df['pri_neigh'] = geoJSON_df['pri_neigh'].str.upper()
geoJSON_df.head()

In [ ]:
map = folium.Map(location=[42.01463639, -87.69015979],tiles = 'cartodbpositron', zoom_start=12, width=800, height=800)


marker_cluster = MarkerCluster().add_to(map)

for point in range(0, len(locationlist)):
    folium.Marker(locationlist[point], popup=sw_cen['a_image_name'][point], 
                  icon=folium.Icon(color='darkblue', icon_color='black', icon='fa-camera', angle=0, prefix='fa')).add_to(marker_cluster)


map

In [ ]:
cen = pd.read_csv("https://raw.githubusercontent.com/gabyakcelik/street_psych/master/ReExtracted_Data_6.7.2021/img_income_comm_geocord_sw.csv")
cen = cen.rename(columns={'Community Area Name' : 'pri_neigh'})
cen

In [ ]:
cen

In [ ]:
geoJSON_df

In [ ]:
w = pd.merge(geoJSON_df, cen, on='pri_neigh')
w

In [ ]:
w = w[['pri_neigh', 'MedianHHIncome', 'geometry']]
w

In [ ]:
new_com_img = pd.read_csv("https://raw.githubusercontent.com/gabyakcelik/street_psych/master/ReExtracted_Data_6.7.2021/MAYBETHISWILLWORK_sw.csv")
new_com_img = new_com_img.rename(columns={'Community Area Name' : 'pri_neigh'})
new_com_img

In [ ]:
df = pd.merge(new_com_img, geoJSON_df,on=['pri_neigh'], how='left')
df = df[['pri_neigh', 'MedianHHIncome','geometry']]
df = gpd.GeoDataFrame(df)
df

In [ ]:
map1 = folium.Map(location=[41.96613547, -87.66107045], tiles = 'cartodbpositron', zoom_start=12, width=800, height=800)




folium.Choropleth( 
    geo_data=w,
    data=w,
    columns=['pri_neigh','MedianHHIncome'],
    key_on='feature.properties.pri_neigh',
    fill_color='BuPu',
    fill_opacity=.5,
    line_opacity=1,
    legend_name="Median Household Income",
    smooth_factor=2.0,
    Highlight= True,
    line_color = "black",
    name = "Median Household Income",
    show=False,
    overlay=True,
    nan_fill_color = "black"
).add_to(map1)


In [ ]:
marker_cluster = MarkerCluster().add_to(map1)

for point in range(0, len(locationlist)):
    folium.Marker(locationlist[point], popup=sw_cen['a_image_name'][point], 
                  icon=folium.Icon(color='darkblue', icon_color='black', icon='fa-camera', angle=0, prefix='fa')).add_to(marker_cluster)


map1

In [ ]:
new_com_img = pd.read_csv("https://raw.githubusercontent.com/gabyakcelik/street_psych/master/ReExtracted_Data_6.7.2021/MAYBETHISWILLWORK_sw.csv")
new_com_img = new_com_img.rename(columns={'Community Area Name' : 'pri_neigh'})
new_com_img

In [ ]:
df = pd.merge(new_com_img, geoJSON_df,on=['pri_neigh'], how='left')
df = df[['pri_neigh', 'MedianHHIncome','geometry']]
df = gpd.GeoDataFrame(df)
df



In [ ]:
folium.Choropleth( 
    geo_data=geoJSON_df,
    data=df,
    columns=['pri_neigh','MedianHHIncome'],
    key_on='feature.properties.pri_neigh',
    fill_color='BuPu',
    fill_opacity=.5,
    line_opacity=1,
    legend_name="Median Household Income",
    smooth_factor=1.0,
    Highlight= True,
    line_color = "black",
    name = "Median Household Income",
    show=False,
    overlay=True,
    nan_fill_color = "black"
).add_to(map)

map

In [ ]:
new_com_img

In [ ]:
w = pd.merge(geoJSON_df, cen, on='pri_neigh')
w

In [ ]:
missing_neigh = np.setdiff1d(geoJSON_neigh, w['sec_neigh'])
missing_neigh

In [ ]:
data = pd.merge(w, geoJSON_df, on=['sec_neigh'])
#data = data.loc[data['MedianHHIncome']!= -666666666]
#data = data[['sec_neigh', 'MedianHHIncome', 'geometry']]
data

missimg = list(data.a_image_name.values)
len(missimg)

data

In [ ]:
imgs = list(w.a_image_name.values)
len(imgs)

In [ ]:
missing_imgs = np.setdiff1d(missimg, imgs)
missing_imgs

In [ ]:
matching_neigh = np.intersect1d(geoJSON_neigh, w['sec_neigh'])
matching_neigh

In [ ]:
df = pd.merge(new_com_img,geoJSON_df, on=['pri_neigh'], how='left')
df = df[['a_image_name','pri_neigh', 'MedianHHIncome', 'geometry']]
df

In [ ]:
final_df = gpd.GeoDataFrame(df)
final_df

In [ ]:
new = pd.merge(new_com_img, sidegeo, on=['a_image_name'])

new = new[['a_image_name', 'MedianHHIncome', 'lat', 'lng']]
new

In [ ]:
new = new[new['MedianHHIncome'] != '-6666666']
new

In [ ]:
imglocations = new[['lat', 'lng']]
imglocations
locationlist = imglocations.values.tolist()
len(locationlist)
locationlist[7]

In [ ]:
comparison_column = np.where(missimg == imgs, True, False)
comparison_column


In [ ]:
for index, i in enumerate(final_df['comp']):
  if i == False: 
    print("CURRENT WORD IS", i, "AT CHARACTER", index)
  

s = final_df.iloc[21]

In [ ]:
final = final_df[['sec_neigh', 'MedianHHIncome','geometry', 'a_image_name']]
final

In [ ]:
final = gpd.GeoDataFrame(df)
final

In [ ]:
map = folium.Map(location=[42.01463639, -87.69015979],tiles = 'cartodbpositron', zoom_start=12)

In [ ]:
map = folium.Map(location=[41.96613547, -87.66107045], tiles = 'cartodbpositron', zoom_start=12, width=800, height=800)


marker_cluster = MarkerCluster().add_to(map)

for point in range(0, len(locationlist)):
    folium.Marker(locationlist[point], popup=final['a_image_name'][point], 
                  icon=folium.Icon(color='darkblue', icon_color='black', icon='fa-camera', angle=0, prefix='fa')).add_to(marker_cluster)


map

In [ ]:
folium.Choropleth( 
    geo_data=geoJSON_df,
    data=final,
    columns=['sec_neigh','MedianHHIncome'],
    key_on='feature.properties.sec_neigh',
    fill_color='BuPu',
    fill_opacity=.5,
    line_opacity=1,
    legend_name="Median Household Income",
    smooth_factor=1.0,
    Highlight= True,
    line_color = "black",
    name = "Median Household Income",
    show=False,
    overlay=True,
    nan_fill_color = "White"
).add_to(map)

map